In [1]:
# details about the relaxed VASP surface, the cell parameters will be taken from here
# I recommend having z be at .5 of l_z so that it is centered in the box
surface_name = 'zigzag'
# this should be a file which is flat in the XYZ plane
molecule_name = 'S_linear'
molecular_shift = [0.0, 0.0, 10.]

# pbe or vdW
functional = 'vdW'  

theta_values = [0,30,45,60,90]

# root directory on remote cluster where jobs will run, no trailing /
rootdir='\/home\/ist001\/work\/bnnt\/surfaces\/rotator'

In [2]:
import numpy as np

from numpy import matrix, sin, cos

import ase.io
import ase.io.vasp

In [3]:
def get_center_of_mass(molecule):
    # techincally this isn't weighted by mass...

    natom = len(molecule)
    Cx = 0.
    Cy = 0.
    Cz = 0.
    for i in range(natom):
        Cx += float(molecule[i][0])
        Cy += float(molecule[i][1])
        Cz += float(molecule[i][2])
    
    com = [Cx/natom, Cy/natom, Cz/natom]
    return com

In [4]:
def write_xyz(species,coordinates,theta):
    outputFile = open('output.xyz','w')
    natom = len(species)
    
    outputFile.write(str(natom) +'\n\n')
    
    for i in range(natom):
        outputFile.write(str(species[i][-2]) + ' ' + str(coordinates[i][0]) + ' ' + str(coordinates[i][1]) + ' ' +  str(coordinates[i][2]) + '\n')
    outputFile.close()    

In [5]:
def rotate_xy(coordinates,theta):
    theta *= np.pi/180.
    rotation = matrix([[cos(theta), -sin(theta), 0.0],
                       [sin(theta),  cos(theta), 0.0],
                       [0.0,         0.0,        1.0]])
    
    coordinates = matrix(coordinates)
    final_vector = np.array(rotation*coordinates.T)
    
    return final_vector.T

In [6]:
def make_run_directory(rootdir, functional, molecule_name, surface_name, molecular_shift, theta):

    shift_string = 'xs.' + str(molecular_shift[0]) + '.ys.' + str(molecular_shift[1]) 
    run_dir = functional + '\/' + molecule_name + '.' + surface_name + '.' + shift_string + '.theta' + str(theta).zfill(2)
# make a run directory (if one doesn't already exist)

    !mkdir -p $run_dir

# copy the non-functional specific stuff to the run directory
    !cp POSCAR resources/POTCAR resources/KPOINTS $run_dir

    if functional == 'vdW':
        !cp resources/INCAR.vdW $run_dir/INCAR 
        !cp resources/vdw_kernel.bindat $run_dir/
    
    else:
        !cp resources/INCAR.pbe $run_dir/INCAR

# write the submit script to the run directory
    !sed s/ROOTDIR/"$rootdir"/g resources/vasp.s | sed s/RUNDIR/"$run_dir"/g > $run_dir/submit.s
    


In [7]:
# file handling stuff
surface_file = 'resources/' + surface_name + '.' + functional + '.CONTCAR'
molecule_file = molecule_name + '.xyz'

surface = ase.io.vasp.read_vasp(surface_file)
surface_cell = surface.get_cell()

# create a temporary XYZ file
tmp_surface = 'surface.xyz'
ase.io.write(tmp_surface, surface)

In [8]:
# optional, leave as 0. otherwise

for theta in theta_values:
    
    molecule_species = np.loadtxt(molecule_file,dtype=str,skiprows=2, usecols=(0,))
    molecule_coords = np.loadtxt(molecule_file,skiprows=2, usecols=(1,2,3))

    # calculate COM before rotation
    center_of_mass = get_center_of_mass(molecule_coords)

    # subtract it off
    molecule_coords -= center_of_mass

    # rotate
    molecule_coords = rotate_xy(molecule_coords, theta)

    # add the shift back on
    molecule_coords += center_of_mass

    # add an optional shift
    molecule_coords += molecular_shift

    # read the substrate
    surface_species = np.loadtxt('surface.xyz',dtype=str,skiprows=2, usecols=(0,))
    surface_coords =  np.loadtxt('surface.xyz',skiprows=2, usecols=(1,2,3))

    combined_coords = np.vstack((molecule_coords, surface_coords))
    combined_species = np.concatenate((molecule_species, surface_species), axis=0)

    write_xyz(combined_species, combined_coords, theta)

    config = ase.io.read('output.xyz')

    config.set_cell(surface_cell)

    comment_line = 'adsorbate = ' + molecule_file + ', surface = ' + surface_file + ', shift = ' + str(molecular_shift) + ', in-plane rotation = ' + str(theta) 

    ase.io.vasp.write_vasp('POSCAR',config,label=comment_line,sort=True, vasp5=True)

    #from ase.visualize import view
    final = ase.io.read('POSCAR')
    #view(final)
    !head POSCAR
    # cleanup
    make_run_directory(rootdir, functional, molecule_name, surface_name, molecular_shift, theta)

#cleanup
!rm $tmp_surface output.xyz
# Tar everything up
!tar cvfz ~/Dropbox/sync.tar.gz $functional/*

adsorbate = S_linear.xyz, surface = resources/zigzag.vdW.CONTCAR, shift = [0.0, 0.0, 10.0], in-plane rotation = 0
 1.0000000000000000
    45.0924602317194143    0.0000000000000000    0.0000000000000000
     0.0000000000000000   22.2001828173343476    0.0000000000000000
     0.0000000000000000    0.0000000000000000   20.4043416806197726
   B   C   H   N   S
 160  14  12 160   3
Cartesian
 10.3552034000000006 17.9357397499999998 10.2021708400000009
  4.4379443199999997 10.2489936999999998 10.2021708400000009
adsorbate = S_linear.xyz, surface = resources/zigzag.vdW.CONTCAR, shift = [0.0, 0.0, 10.0], in-plane rotation = 30
 1.0000000000000000
    45.0924602317194143    0.0000000000000000    0.0000000000000000
     0.0000000000000000   22.2001828173343476    0.0000000000000000
     0.0000000000000000    0.0000000000000000   20.4043416806197726
   B   C   H   N   S
 160  14  12 160   3
Cartesian
 10.3552034000000006 17.9357397499999998 10.2021708400000009
  4.4379443199999997 10.248993699999